# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
import math
import scipy.stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
from api_secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [5]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

[{'avgTotalVolume': 63853142, 'calculationPrice': 'close', 'change': 2.29, 'changePercent': 0.01222, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 189.73, 'iexCloseTime': 1715803197984, 'iexLastUpdated': 0, 'iexMarketPercent': None, 'iexOpen': None, 'iexOpenTime': None, 'iexRealtimePrice': 0, 'iexRealtimeSize': 0, 'iexVolume': 0, 'lastTradeTime': 1715803199995, 'latestPrice': 189.72, 'latestSource': 'Close', 'latestTime': 'May 15, 2024', 'latestUpdate': 1715803200426, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2909182037040, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'open': None, 'open

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [6]:
price = data[0]["latestPrice"]
peRatio = data[0]["peRatio"]
print(price, peRatio)

189.72 29.51


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns= my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
print(data[0])

{'avgTotalVolume': 2215734, 'calculationPrice': 'close', 'change': 0.85, 'changePercent': 0.00618, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Yum Brands Inc.', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': '15 minute delayed price', 'highTime': 1715803199978, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 138.31, 'iexCloseTime': 1715803195417, 'iexLastUpdated': 0, 'iexMarketPercent': None, 'iexOpen': None, 'iexOpenTime': None, 'iexRealtimePrice': 0, 'iexRealtimeSize': 0, 'iexVolume': 0, 'lastTradeTime': 1715803199714, 'latestPrice': 138.34, 'latestSource': 'Close', 'latestTime': 'May 15, 2024', 'latestUpdate': 1715803202371, 'latestVolume': None, 'low': None, 'lowSource': '15 minute delayed price', 'lowTime': 1715779800747, 'marketCap': 38961000208, 'oddLotDelayedPri

In [9]:
final_dataframe = pd.DataFrame(columns= my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for position in range(0,len(symbol_string.split(','))):
        final_dataframe.loc[len(final_dataframe.index)] = [data[position]["symbol"], data[position]["latestPrice"],data[position]['peRatio'],'N/A']

final_dataframe

C:\Users\Manu\AppData\Local\Temp\ipykernel_8852\3924831661.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe.loc[len(final_dataframe.index)] = [data[position]["symbol"], data[position]["latestPrice"],data[position]['peRatio'],'N/A']
C:\Users\Manu\AppData\Local\Temp\ipykernel_8852\3924831661.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe.loc[len(final_dataframe.index)] = [data[position]["symbol"], data[position]["latestPrice"],data[position]['peRatio'],'N/A'

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,154.08,36.77,N/A
1,AAL,14.94,22.79,N/A
2,AAP,75.86,152.09,N/A
3,AAPL,189.72,29.51,N/A
4,ABBV,163.79,48.75,N/A
...,...,...,...,...
500,YUM,138.34,24.48,N/A
501,ZBH,120.98,26.19,N/A
502,ZBRA,323.87,64.13,N/A
503,ZION,45.10,11.30,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [10]:
final_dataframe.sort_values("Price-to-Earnings Ratio",ascending = False, inplace=True)
final_dataframe = final_dataframe[final_dataframe["Price-to-Earnings Ratio"] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe = final_dataframe.reset_index(drop = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,KSU,293.59,279.61,N/A
1,AMD,159.67,233.50,N/A
2,GILD,67.01,173.69,N/A
3,VNO,25.42,163.05,N/A
4,AAP,75.86,152.09,N/A
5,MRK,131.73,145.40,N/A
6,IRM,82.34,125.19,N/A
7,WELL,101.00,123.40,N/A
8,LLY,787.02,115.91,N/A
9,ARE,125.17,115.90,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
        portfolio_size = val
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = float(input("Enter the value of your portfolio:"))

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [12]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [13]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[row,'Price'])


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [14]:
symbol = 'AAPL'
batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote,advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
                                                     #1º Quote, 2º Advanced_stats, 31 Fundamental Valuations

# Price-to-earnings ratio
pe_ratio = data[0]['peRatio']
print(pe_ratio)

# Price-to-book ratio  // ADVANCED_STATS Endpoint
pb_ratio = data[1]["priceToBook"]
print(pb_ratio)

# Price-to-sales ratio  // ADVANCED_STATS Endpoint
ps_ratio = data[1]["priceToSales"]
print(ps_ratio)

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[1]['enterpriseValue']
ebitda = data[1]['EBITDA']

ev_to_ebitda = enterprise_value/ebitda
print(ev_to_ebitda)

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[1]['grossProfit']

ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)

29.51
39.21
7.62
22.731194694397086
16.93791911660899


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [15]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile', 
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote,advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for position in range(0,len(symbol_string.split(','))):
        enterprise_value = data[position + int(len(data)/2)]['enterpriseValue']
        ebitda = data[position + int(len(data)/2)]['EBITDA']
        gross_profit = data[position + int(len(data)/2)]['grossProfit']
        try:
            ev_to_ebitda_ratio = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda_ratio = np.NaN

        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe.loc[len(rv_dataframe.index)] = [
            data[position]["symbol"], 
            data[position]["latestPrice"],
            'N/A',
            data[position]['peRatio'],
            'N/A',
            data[position + int(len(data)/2)]["priceToBook"],
            'N/A',
            data[position + int(len(data)/2)]["priceToSales"],
            'N/A',
            ev_to_ebitda_ratio,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
            ]

rv_dataframe


C:\Users\Manu\AppData\Local\Temp\ipykernel_8852\3762176656.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rv_dataframe.loc[len(rv_dataframe.index)] = [
C:\Users\Manu\AppData\Local\Temp\ipykernel_8852\3762176656.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rv_dataframe.loc[len(rv_dataframe.index)] = [
C:\Users\Manu\AppData\Local\Temp\ipykernel_8852\3762176656.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.08,N/A,36.77,N/A,7.30,N/A,6.7000,N/A,29.200100,N/A,13.498079,N/A,N/A
1,AAL,14.94,N/A,22.79,N/A,-1.78,N/A,0.1844,N/A,6.094756,N/A,0.636310,N/A,N/A
2,AAP,75.86,N/A,152.09,N/A,1.79,N/A,0.4006,N/A,13.793814,N/A,1.283268,N/A,N/A
3,AAPL,189.72,N/A,29.51,N/A,39.21,N/A,7.6200,N/A,22.731195,N/A,16.937919,N/A,N/A
4,ABBV,163.79,N/A,48.75,N/A,36.12,N/A,5.3200,N/A,15.650124,N/A,10.187824,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.34,N/A,24.48,N/A,-5.02,N/A,5.5400,N/A,20.227626,N/A,9.413655,N/A,N/A
501,ZBH,120.98,N/A,26.19,N/A,1.98,N/A,3.3400,N/A,14.836764,N/A,5.654593,N/A,N/A
502,ZBRA,323.87,N/A,64.13,N/A,5.26,N/A,3.8200,N/A,29.828849,N/A,9.219020,N/A,N/A
503,ZION,45.10,N/A,11.30,N/A,1.24,N/A,1.4000,N/A,7.061748,N/A,1.431784,N/A,N/A


In [16]:
try:
    ev_to_ebitda_ratio = enterprise_value/ebitda
except TypeError:
    ev_to_ebitda_ratio = np.NaN

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,381.020,N/A,65.69,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,182.500,N/A,59.45,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,287.790,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
48,ATVI,94.420,N/A,34.59,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,94.920,N/A,47.94,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.73,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,103.900,N/A,39.96,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,24.420,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
137,DISH,5.770,N/A,2.90,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.260,N/A,14.27,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

---

If you do it with all the columns it throws you an error due to the initial column that is made of strings rather than floats.

Here is the code to do this:

In [18]:
# rv_dataframe.columns                  // To get the column which we want to iterate
subset_columns = ['Price', 'Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']

for position in range(0,len(subset_columns)):
    # print(position)
    # print(rv_dataframe[subset_columns[position]])
    rv_dataframe[subset_columns[position]].fillna(rv_dataframe[subset_columns[position]].mean(), inplace= True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [19]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

---

It is made on purpose with different names in order to iterate it with a dictionary.

**NOTE**: When we are using the for loop with the keys, we are moving through the columns that belongs to the Percentile side.

In [33]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:    #Objetivo: Recorrer una columna de arriba abajo
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric],rv_dataframe.loc[row,metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,14245,0.43,0.16,0.0456,0.08,0.0934,0.02,0.270279,0.02,0.094878,0.02,3.366337
1,LNC,29.97,1668,3.80,0.18,0.7768,0.18,0.3134,0.2,2.900230,0.06,0.313427,0.04,4.435644
2,KSS,25.66,1948,8.97,0.44,0.7310,0.12,0.1628,0.04,4.881220,0.22,1.025490,0.3,5.980198
3,GM,45.49,1099,5.60,0.2,0.7793,0.2,0.2968,0.18,1.884552,0.04,1.344652,0.52,6.019802
4,UNM,52.48,952,7.79,0.38,0.9725,0.3,0.7919,0.58,4.567058,0.18,0.791932,0.14,8.079208
5,CVS,56.26,888,9.87,0.52,0.9548,0.28,0.1957,0.08,6.922419,0.56,0.826539,0.2,8.079208
6,XRX,14.15,3533,-10.11,0.1,0.7390,0.14,0.2634,0.12,9.240328,0.86,1.081873,0.34,8.435644
7,UA,6.66,7507,7.49,0.3,1.2400,0.6,0.4673,0.37,5.649607,0.37,0.884035,0.25,8.534653
8,UAA,6.80,7352,7.64,0.34,1.2400,0.6,0.4673,0.37,5.649607,0.37,0.884035,0.25,8.613861
9,SYF,44.14,1132,6.36,0.24,1.2600,0.65,0.7311,0.52,4.133428,0.14,1.002270,0.28,8.752475


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [34]:
from statistics import mean
mean([5,10])

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,14245,0.43,0.16,0.0456,0.08,0.0934,0.02,0.270279,0.02,0.094878,0.02,0.06
1,LNC,29.97,1668,3.80,0.18,0.7768,0.18,0.3134,0.2,2.900230,0.06,0.313427,0.04,0.132
2,KSS,25.66,1948,8.97,0.44,0.7310,0.12,0.1628,0.04,4.881220,0.22,1.025490,0.3,0.224
3,GM,45.49,1099,5.60,0.2,0.7793,0.2,0.2968,0.18,1.884552,0.04,1.344652,0.52,0.228
4,UNM,52.48,952,7.79,0.38,0.9725,0.3,0.7919,0.58,4.567058,0.18,0.791932,0.14,0.316
5,CVS,56.26,888,9.87,0.52,0.9548,0.28,0.1957,0.08,6.922419,0.56,0.826539,0.2,0.328
6,XRX,14.15,3533,-10.11,0.1,0.7390,0.14,0.2634,0.12,9.240328,0.86,1.081873,0.34,0.312
7,UA,6.66,7507,7.49,0.3,1.2400,0.6,0.4673,0.37,5.649607,0.37,0.884035,0.25,0.378
8,UAA,6.80,7352,7.64,0.34,1.2400,0.6,0.4673,0.37,5.649607,0.37,0.884035,0.25,0.386
9,SYF,44.14,1132,6.36,0.24,1.2600,0.65,0.7311,0.52,4.133428,0.14,1.002270,0.28,0.366


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [35]:
rv_dataframe.sort_values('RV Score', ascending= True, inplace= True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

C:\Users\Manu\AppData\Local\Temp\ipykernel_8852\1781097109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('RV Score', ascending= True, inplace= True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,14245,0.43,0.16,0.0456,0.08,0.0934,0.02,0.270279,0.02,0.094878,0.02,0.06
1,LNC,29.97,1668,3.80,0.18,0.7768,0.18,0.3134,0.2,2.900230,0.06,0.313427,0.04,0.132
2,KSS,25.66,1948,8.97,0.44,0.7310,0.12,0.1628,0.04,4.881220,0.22,1.025490,0.3,0.224
3,GM,45.49,1099,5.60,0.2,0.7793,0.2,0.2968,0.18,1.884552,0.04,1.344652,0.52,0.228
4,XRX,14.15,3533,-10.11,0.1,0.7390,0.14,0.2634,0.12,9.240328,0.86,1.081873,0.34,0.312
5,UNM,52.48,952,7.79,0.38,0.9725,0.3,0.7919,0.58,4.567058,0.18,0.791932,0.14,0.316
6,DXC,19.73,2534,-8.69,0.12,1.1600,0.48,0.2601,0.1,3.575879,0.1,2.055391,0.8,0.32
7,CVS,56.26,888,9.87,0.52,0.9548,0.28,0.1957,0.08,6.922419,0.56,0.826539,0.2,0.328
8,AAL,14.94,3346,22.79,1.0,-1.7800,0.06,0.1844,0.06,6.094756,0.42,0.636310,0.12,0.332
9,UAL,54.96,909,6.79,0.26,1.9700,0.93,0.3296,0.26,3.899195,0.12,0.570723,0.1,0.334


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [36]:
portfolio_input()

In [37]:
position_size = portfolio_size/len(rv_dataframe.index)
position_size

for row in rv_dataframe.index:
    rv_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(position_size/rv_dataframe.loc[row,"Price"])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.51,14245,0.43,0.16,0.0456,0.08,0.0934,0.02,0.270279,0.02,0.094878,0.02,0.06
1,LNC,29.97,1668,3.80,0.18,0.7768,0.18,0.3134,0.2,2.900230,0.06,0.313427,0.04,0.132
2,KSS,25.66,1948,8.97,0.44,0.7310,0.12,0.1628,0.04,4.881220,0.22,1.025490,0.3,0.224
3,GM,45.49,1099,5.60,0.2,0.7793,0.2,0.2968,0.18,1.884552,0.04,1.344652,0.52,0.228
4,XRX,14.15,3533,-10.11,0.1,0.7390,0.14,0.2634,0.12,9.240328,0.86,1.081873,0.34,0.312
5,UNM,52.48,952,7.79,0.38,0.9725,0.3,0.7919,0.58,4.567058,0.18,0.791932,0.14,0.316
6,DXC,19.73,2534,-8.69,0.12,1.1600,0.48,0.2601,0.1,3.575879,0.1,2.055391,0.8,0.32
7,CVS,56.26,888,9.87,0.52,0.9548,0.28,0.1957,0.08,6.922419,0.56,0.826539,0.2,0.328
8,AAL,14.94,3346,22.79,1.0,-1.7800,0.06,0.1844,0.06,6.094756,0.42,0.636310,0.12,0.332
9,UAL,54.96,909,6.79,0.26,1.9700,0.93,0.3296,0.26,3.899195,0.12,0.570723,0.1,0.334


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [38]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name= 'Value Strategy', index = False)


## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [39]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [40]:
column_formats = {
        'A':['Ticker', string_template],
        'B':['Price', dollar_template],
        'C':['Number of Shares to Buy', integer_template],
        'D':['Price-to-Earnings Ratio', float_template],
        'E':['PE Percentile', percent_template],
        'F':['Price-to-Book Ratio', float_template],
        'G':['PB Percentile', percent_template], 
        'H':['Price-to-Sales Ratio', float_template],
        'I':['PS Percentile', percent_template],
        'J':['EV/EBITDA', float_template],
        'K':['EV/EBITDA Percentile', percent_template],
        'L':['EV/GP', float_template],
        'M':['EV/GP Percentile', percent_template],
        'N':['RV Score', percent_template]
    }

for column in column_formats.keys():
    writer.sheets["Value Strategy"].set_column(f'{column}:{column}',25, column_formats[column][1])
    writer.sheets["Value Strategy"].write(f'{column}1',column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [41]:
writer.close()